# 17-Class Flower Image Classifier 

In [ ]:
# Import all necessary packages 
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.applications import vgg16, inception_v3, resnet50, mobilenet, vgg19
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.vgg19 import preprocess_input 
from keras.applications.mobilenet import preprocess_input
from keras.applications.inception_v3 import preprocess_input

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2


In [ ]:
# Modified AlexNet
def modifiedAlexnet(width, height, depth, classes):

    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(width,height,depth), kernel_size=(11,11),
                     strides=(4,4), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Passing it to a dense layer
    model.add(Flatten())
    # 1st Dense Layer
    model.add(Dense(4096, input_shape=(width*height*depth,)))
    model.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 2nd Dense Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Dense Layer
    model.add(Dense(1000))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model

In [ ]:
# Add full path of dataset 
data_dir = 'flowers17'
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

In [ ]:
# Batch size
batch_size = 128
# Number of Epochs
epochs = 75

# Data preprocessing and Data Augmentation

datagen_train = ImageDataGenerator(rescale=1./255)

datagen_test = ImageDataGenerator(rescale=1./255)

generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    batch_size=batch_size,
                                                    target_size=(224,224),
                                                    shuffle = True,
                                                    class_mode = 'categorical')

generator_test = datagen_test.flow_from_directory(  directory=test_dir,
                                                    batch_size=batch_size,
                                                    target_size=(224,224),
                                                    class_mode = 'categorical',
                                                    shuffle = False)

In [ ]:
# Number of steps per test and number of steps per epoch
steps_test = generator_test.n // batch_size
print("\nNumber of steps per test :",steps_test)

steps_per_epoch = generator_train.n // batch_size
print("\nNumber of steps per epoch:",steps_per_epoch)


In [ ]:
# MODEL 
model = modifiedALextnet(224,224,3,17)

# Model Summary

model.summary()

In [ ]:
# Using Adam & rmrprop Optimizer 
#rms = keras.optimizers.RMSprop(lr=0.0001,epsilon=None, decay=0.0)
Adam = keras.optimizers.adam(lr=0.001,beta_1=0.9,amsgrad = True)

# Compile the Model 
# Using categorical crossentropy and accuracy for metrics
model.compile(loss='categorical_crossentropy', optimizer= Adam, metrics=['accuracy'])

# Fit the model 
history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)

In [ ]:
plt.figure(figsize=[20,20])
plt.plot(history.history['loss'],'r',linewidth=5.0)
plt.plot(history.history['val_loss'],'b',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=25)
plt.ylabel('Loss',fontsize=25)
plt.title('Loss Curves',fontsize=25)

plt.figure(figsize=[20,20])
plt.plot(history.history['acc'],'r',linewidth=5.0)
plt.plot(history.history['val_acc'],'b',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=25)
plt.ylabel('Accuracy',fontsize=25)
plt.title('Accuracy Curves',fontsize=25)

# Data augmentation and preprocessing 

In [ ]:
# Batch size
batch_size = 64
# Number of Epochs
epochs = 75

# Data preprocessing and Data Augmentation

datagen_train = ImageDataGenerator(shear_range =0.2 ,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rescale=1./255)

datagen_test = ImageDataGenerator(rescale=1./255)

generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    batch_size=batch_size,
                                                    target_size=(224,224),
                                                    shuffle = True,
                                                    class_mode = 'categorical')

generator_test = datagen_test.flow_from_directory(  directory=test_dir,
                                                    batch_size=batch_size,
                                                    target_size=(224,224),
                                                    class_mode = 'categorical',
                                                    shuffle = False)

In [ ]:
# Using Adam Optimizer 
Adam = keras.optimizers.adam(lr=0.001,beta_1=0.9,amsgrad = True)

# Compile the Model 
# Using categorical crossentropy and accuracy for metrics
model.compile(loss='categorical_crossentropy', optimizer= Adam, metrics=['accuracy'])

# Fit the model 
history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)

In [ ]:
plt.figure(figsize=[20,20])
plt.plot(history.history['loss'],'k',linewidth=5.0)
plt.plot(history.history['val_loss'],'b',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=25)
plt.ylabel('Loss',fontsize=25)
plt.title('Loss Curves',fontsize=25)

plt.figure(figsize=[20,20])
plt.plot(history.history['acc'],'k',linewidth=5.0)
plt.plot(history.history['val_acc'],'b',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=25)
plt.ylabel('Accuracy',fontsize=25)
plt.title('Accuracy Curves',fontsize=25)

# Training on pre-trained network : Transfer Learning : Fine Tuning top layers

In [ ]:
base_model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
predictions = Dense(17, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs = predictions)


for layer in base_model.layers:
    
    layer.trainable = False 

    
              
optimizer = keras.optimizers.Adam(lr=0.0001,beta_1=0.9,amsgrad=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)


In [ ]:
# Plot showing training loss and validation loss
plt.figure(figsize=[15,15])
plt.plot(history.history['loss'],'r',linewidth=5.0)
plt.plot(history.history['val_loss'],'y',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.title('Loss Curves using Inception_V3',fontsize=20)


# Plot showing training accuracy and validation accuracy 
plt.figure(figsize=[15,15])
plt.plot(history.history['acc'],'r',linewidth=5.0)
plt.plot(history.history['val_acc'],'y',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.title('Accuracy Curves using Inception_V3',fontsize=20)


In [ ]:
base_model = resnet50.ResNet50(weights = 'imagenet', include_top = False, input_shape = (224,224,3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
predictions = Dense(17, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs = predictions)


for layer in base_model.layers:
    
    layer.trainable = False 

   
              
optimizer = keras.optimizers.Adam(lr=0.0001,beta_1=0.9,amsgrad=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)


In [ ]:
# Plot showing training loss and validation loss
plt.figure(figsize=[15,15])
plt.plot(history.history['loss'],'r',linewidth=5.0)
plt.plot(history.history['val_loss'],'y',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.title('Loss Curves using ResNet50',fontsize=20)


# Plot showing training accuracy and validation accuracy 
plt.figure(figsize=[15,15])
plt.plot(history.history['acc'],'r',linewidth=5.0)
plt.plot(history.history['val_acc'],'y',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.title('Accuracy Curves using ResNet50',fontsize=20)


In [ ]:
base_model = mobilenet.MobileNet(input_shape=(224,224,3),include_top= False, weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
predictions = Dense(17, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs = predictions)


for layer in base_model.layers:
    
    layer.trainable = False 

   
              
optimizer = keras.optimizers.Adam(lr=0.0001,beta_1=0.9,amsgrad=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)


In [ ]:
# Plot showing training loss and validation loss
plt.figure(figsize=[15,15])
plt.plot(history.history['loss'],'r',linewidth=5.0)
plt.plot(history.history['val_loss'],'y',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.title('Loss Curves using mobileNet',fontsize=20)


# Plot showing training accuracy and validation accuracy 
plt.figure(figsize=[15,15])
plt.plot(history.history['acc'],'r',linewidth=5.0)
plt.plot(history.history['val_acc'],'y',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.title('Accuracy Curves using mobileNet',fontsize=20)


In [ ]:
base_model = keras.applications.xception.Xception(include_top=False, weights='imagenet',input_shape=None)


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
predictions = Dense(17, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs = predictions)


for layer in base_model.layers:
    
    layer.trainable = False 

   
              
optimizer = keras.optimizers.Adam(lr=0.0001,beta_1=0.9,amsgrad=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)


In [ ]:
# Plot showing training loss and validation loss
plt.figure(figsize=[15,15])
plt.plot(history.history['loss'],'r',linewidth=5.0)
plt.plot(history.history['val_loss'],'y',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.title('Loss Curves using xception',fontsize=20)


# Plot showing training accuracy and validation accuracy 
plt.figure(figsize=[15,15])
plt.plot(history.history['acc'],'r',linewidth=5.0)
plt.plot(history.history['val_acc'],'y',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.title('Accuracy Curves using xception',fontsize=20)


In [ ]:
base_model = vgg19.VGG19(weights = 'imagenet', include_top = False)


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
predictions = Dense(17, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs = predictions)


for layer in base_model.layers:
    
    layer.trainable = False 

              
optimizer = keras.optimizers.Adam(lr=0.0001,beta_1=0.9,amsgrad=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)


In [ ]:
# Plot showing training loss and validation loss
plt.figure(figsize=[15,15])
plt.plot(history.history['loss'],'r',linewidth=5.0)
plt.plot(history.history['val_loss'],'y',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.title('Loss Curves using VGG19',fontsize=20)


# Plot showing training accuracy and validation accuracy 
plt.figure(figsize=[15,15])
plt.plot(history.history['acc'],'r',linewidth=5.0)
plt.plot(history.history['val_acc'],'y',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.title('Accuracy Curves using VGG19',fontsize=20)


In [ ]:
base_model = vgg16.VGG16(weights = 'imagenet', include_top = False)


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
predictions = Dense(17, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs = predictions)


for layer in base_model.layers:
    
    layer.trainable = False 
              
optimizer = keras.optimizers.Adam(lr=0.0001,beta_1=0.9,amsgrad=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

history= model.fit_generator(generator_train,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_data = generator_test,
                           validation_steps = steps_test)


In [ ]:
# Plot showing training loss and validation loss
plt.figure(figsize=[15,15])
plt.plot(history.history['loss'],'b',linewidth=5.0)
plt.plot(history.history['val_loss'],'k',linewidth=5.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.title('Loss Curves using VGG_16',fontsize=20)


# Plot showing training accuracy and validation accuracy 
plt.figure(figsize=[15,15])
plt.plot(history.history['acc'],'b',linewidth=5.0)
plt.plot(history.history['val_acc'],'k',linewidth=5.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=25)
plt.xlabel('Epochs ',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.title('Accuracy Curves using VGG_16',fontsize=20)
